<a href="https://colab.research.google.com/github/imdoamaral/imersao_dev_agentes_de_IA_alura_2025/blob/main/imersao_agentes_de_IA_alura_%2B_google_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 1

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [3]:
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [4]:
resp_test = llm.invoke('Quem é você?')

In [5]:
print(resp_test.content)

Eu sou um modelo de linguagem grande, treinado pelo Google.


In [6]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [7]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal['AUTO_RESOLVER', 'PEDIR_INFO', 'ABRIR_CHAMADO']
  urgencia: Literal['BAIXA', 'MEDIA', 'ALTA']
  campos_faltantes: List[str] = Field(default_factory=list)

In [8]:
llm_triagem = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [9]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
      ])

  return saida.model_dump()


In [10]:
testes = [
    'Posso reembolsar a internet?',
    'Quero mais 5 dias de trabalho remoto. Como faço?',
    'Posso reembolsar cursos ou treinamentos da Alura?',
    'Quantas capivaras tem no Rio Piracicaba?'
]

In [11]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Piracicaba?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['informacao_relevante_a_politica']}



# Aula 2

In [13]:
!pip install -q --upgrade langchain_community faiss_cpu langchain-text-splitters pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [14]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path('/content/').glob('*.pdf'):
  try:
    loader = PyMuPDFLoader(str(n))
    docs.extend(loader.load())
    print(f'Arquivo carreado com sucesso: {n}')
  except Exception as e:
    print(f'Erro ao carregar o arquivo {n}: {e}')

print(f'Total de documentos carregados: {len(docs)}')

Arquivo carreado com sucesso: /content/Políticas de Home Office.pdf
Arquivo carreado com sucesso: /content/Política de Reembolsos (Viagens e Despesas).pdf
Arquivo carreado com sucesso: /content/Política de Uso de E-mail e Segurança da Informação.pdf
Total de documentos carregados: 3


In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = splitter.split_documents(docs)

In [19]:
for chunk in chunks:
  print(chunk)
  print('----------------------------------------\n')

page_content='Políticas de Home Office 
 
1.​ A empresa adota modelo híbrido: mínimo de 2 dias presenciais por semana, salvo 
exceções aprovadas pelo gestor e RH.​
 
2.​ Equipamentos: a empresa fornece notebook e periféricos. O colaborador é 
responsável por zelar pela conservação.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Políticas de Home Office.pdf', 'file_path': '/content/Políticas de Home Office.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Políticas de Home Office', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
----------------------------------------

page_content='3.​ Segurança: é obrigatório uso de VPN e bloqueio de tela. Documentos confidenciais 
não devem ser impressos fora do escritório.​
 
4.​ Ergonomia: recomendamos cadeira adequada e suporte de monitor. O RH pode 
avaliar solicitação de apoio ergonômico.​' metadata

In [22]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model='models/gemini-embedding-001',
    google_api_key=GOOGLE_API_KEY
)

In [28]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.3, 'k':4}
)

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ('system',
    'Você é um assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento.'
    'Responda SOMENTE com base no contexto fornecido.'
    'Se não houver base suficiente, responda apenas "Não sei".'),

    ('human', 'Pergunta: {input}\n\nContexto:\n{context}')
])

document_chain = create_stuff_documents_chain(
    llm_triagem,
    prompt_rag,
)

In [34]:
# Formatadores
import re, pathlib
from typing import List, Dict

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [35]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
  docs_relacionados = retriever.invoke(pergunta)

  if not docs_relacionados:
      return {"answer": "Não sei.",
              "citacoes": [],
              "contexto_encontrado": False}

  answer = document_chain.invoke({"input": pergunta,
                                  "context": docs_relacionados})

  txt = (answer or "").strip()

  if txt.rstrip(".!?") == "Não sei":
      return {"answer": "Não sei.",
              "citacoes": [],
              "contexto_encontrado": False}

  return {"answer": txt,
          "citacoes": formatar_citacoes(docs_relacionados, pergunta),
          "contexto_encontrado": True}

In [32]:
testes = [
    'Posso reembolsar a internet?',
    'Quero mais 5 dias de trabalho remoto. Como faço?',
    'Posso reembolsar cursos ou treinamentos da Alura?',
    'Quantas capivaras tem no Rio Piracicaba?'
]

In [36]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------------------------------")

PERGUNTA: Posso reembolsar a internet?
RESPOSTA: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
CITAÇÕES:
 - Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
   Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
 - Documento: Políticas de Home Office.pdf, Página: 1
   Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de
------------------------------------------------------------
PERGUNTA: Quero mais 5 dias de trabalho remoto. Como faço?
RESPOSTA: Você deve formalizar a solicitação via chamado ao RH, incluindo a justificativa do seu gestor.
CITAÇÕES:
 - Documento: Políticas de Home Office.pdf, Página: 